# homework Q1. Running Elastic

"build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",

In [56]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url) #request macht runterladen
documents_raw = docs_response.json() #

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)



# homework: Q2. Indexing the data


In [ ]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

In [ ]:
from tqdm.auto import tqdm    #damit progress-bar angezeigt wird damit man ein gefühl hat wie lange es noch dauert

/opt/homebrew/Caskroom/miniforge/base/envs/llm-zoomcamp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
for doc in tqdm(documents):  #jetzt werden die dokumente geindext
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:01<00:00, 637.46it/s]


Which function do you use for adding your data to elastic?

* Answer: index


# homework Q3. Searching


In [58]:
query = 'How do I execute a command in a running docker container?'

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
    
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)
    
result_docs = []
    
for hit in response['hits']['hits']:
    result_docs.append(hit['_score'])

result_docs[0]


84.050095

In [59]:
response['hits']['hits']

[{'_index': 'course-questions',
  '_id': 'Y9htRpABF5qcBq5__mt3',
  '_score': 84.050095,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'cdhtRpABF5qcBq5_-2mF',
  '_score': 75.54128,
  '_source': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\n

What's the score for the top ranking result?
* Answer: 84.05


# homework: Q4. Filtering


In [60]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    } #filter: schränkt ein auf Ergebnisse die in Kurs-Feld "data-engineering-zoomcamp" sind

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

query = 'How do I execute a command in a running docker container?'


In [61]:
import pandas as pd
response = elastic_search(query)
pd.DataFrame(response)


,text,section,question,course
0,Launch the container image in interactive mode...,5. Deploying Machine Learning Models,How do I debug a docker container?,machine-learning-zoomcamp
1,You can copy files from your local machine int...,5. Deploying Machine Learning Models,How do I copy files from my local machine to d...,machine-learning-zoomcamp
2,You can copy files from your local machine int...,5. Deploying Machine Learning Models,How do I copy files from a different folder in...,machine-learning-zoomcamp


In [62]:
elastic_search(query)[2]['question']

'How do I copy files from a different folder into docker container’s working directory?'

# homework: Q5. Building a prompt


In [63]:
context = ""

for doc in response:
    context = context + f"Q: {doc['question']}\nA: {doc['text']}".strip() + "\n\n"

In [64]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

prompt = prompt_template.format(question=query, context=context).strip()


In [65]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

In [66]:
print(len(prompt))

1462


# homework: Q6. Tokens

In [67]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")
tokens=encoding.encode(prompt)
print(tokens)

[63842, 261, 4165, 14029, 29186, 13, 30985, 290, 150339, 4122, 402, 290, 31810, 8099, 591, 290, 40251, 7862, 558, 8470, 1606, 290, 19719, 591, 290, 31810, 8099, 1261, 55959, 290, 150339, 364, 107036, 25, 3253, 621, 357, 15792, 261, 6348, 306, 261, 6788, 62275, 9282, 1715, 10637, 50738, 734, 48, 25, 3253, 621, 357, 15199, 261, 62275, 9282, 3901, 32, 25, 41281, 290, 9282, 3621, 306, 25383, 6766, 326, 151187, 290, 7251, 4859, 11, 813, 484, 480, 13217, 261, 38615, 6348, 558, 68923, 2461, 533, 278, 2230, 7962, 4859, 38615, 464, 3365, 523, 3335, 290, 9282, 382, 4279, 6788, 11, 15792, 261, 6348, 306, 290, 4857, 9282, 734, 68923, 10942, 350, 6555, 290, 9282, 26240, 446, 68923, 25398, 533, 278, 464, 6896, 26240, 29, 38615, 198, 6103, 277, 10732, 391, 79771, 1029, 48, 25, 3253, 621, 357, 5150, 6291, 591, 922, 2698, 7342, 316, 62275, 9282, 3901, 32, 25, 1608, 665, 5150, 6291, 591, 634, 2698, 7342, 1511, 261, 91238, 9282, 2360, 290, 62275, 27776, 6348, 13, 44257, 1495, 316, 621, 480, 734, 1385, 51

In [ ]:
len(tokens)

322

In [68]:
decode_list=[63842, 261, 4165, 14029, 29186, 13, 30985, 290, 150339, 4122, 402, 290, 31810, 8099, 591, 290, 40251, 7862, 558, 8470, 1606]
for element in decode_list:
    print(encoding.decode_single_token_bytes(element))


b"You're"
b' a'
b' course'
b' teaching'
b' assistant'
b'.'
b' Answer'
b' the'
b' QUESTION'
b' based'
b' on'
b' the'
b' CONT'
b'EXT'
b' from'
b' the'
b' FAQ'
b' database'
b'.\n'
b'Use'
b' only'


In [69]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

# homework: bonus

Bonus: calculating the costs (ungraded)
Suppose that on average per request we send 150 tokens and receive back 250 tokens.

How much will it cost to run 1000 requests?

You can see the prices here

On June 17, the prices for gpt4o are:

Input: $0.005 / 1K tokens
Output: $0.015 / 1K tokens
You can redo the calculations with the values you got in Q6 and Q7.

In [70]:
print(250*0.005+150*0.015)

3.5
